# Token classification (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 4.7 MB/s 
     |████████████████████████████████| 69 kB 9.5 MB/s 
     |████████████████████████████████| 5.3 MB 62.9 MB/s 
     |████████████████████████████████| 163 kB 52.5 MB/s 
     |████████████████████████████████| 115 kB 68.3 MB/s 
     |████████████████████████████████| 212 kB 71.4 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
     |████████████████████████████████| 7.6 MB 31.2 MB/s 
     |████████████████████████████████| 1.3 MB 41.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 148 kB 4.7 MB/s 
Reading package lists... Done
Building dependency tree       
Readi

You will need to setup git, adapt your email and name in the following cell.

In [2]:
#!git config --global user.email "mariafogh@gmail.com"
#!git config --global user.name "MariaFogh"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset('wikiann', 'en')

Dataset wikiann downloaded and prepared to /zhome/a6/6/127219/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 353.82it/s]


In [4]:
raw_datasets

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [5]:
raw_datasets["train"][0]["tokens"]

['R.H.',
 'Saunders',
 '(',
 'St.',
 'Lawrence',
 'River',
 ')',
 '(',
 '968',
 'MW',
 ')']

In [6]:
raw_datasets["train"][0]["ner_tags"]

[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]

In [7]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

In [8]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [9]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

R.H.  Saunders ( St.   Lawrence River ) ( 968 MW ) 
B-ORG I-ORG    O B-ORG I-ORG    I-ORG O O O   O  O 


In [10]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 23.9kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 451kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 505kB/s] 
Downloading: 100%|██████████| 436k/436k [00:00<00:00, 826kB/s]  


In [11]:
tokenizer.is_fast

True

In [12]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'R',
 '.',
 'H',
 '.',
 'Saunders',
 '(',
 'St',
 '.',
 'Lawrence',
 'River',
 ')',
 '(',
 '96',
 '##8',
 'MW',
 ')',
 '[SEP]']

In [13]:
inputs.word_ids()

[None, 0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]

In [14]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [15]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]
[-100, 3, 4, 4, 4, 4, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, -100]


In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [17]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f92ee02e7a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 20/20 [00:03<00:00,  5.84ba/s]


In [18]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [19]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    3,    4,    4,    4,    4,    0,    3,    4,    4,    4,    0,
            0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    1,    2,    2,    2,    0,    0,    0,
         -100, -100, -100, -100, -100, -100]])

In [20]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 4, 4, 4, 4, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, -100]


In [22]:
import evaluate

metric = evaluate.load("seqeval")

In [23]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O']

In [24]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [25]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [26]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [27]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Downloading: 100%|██████████| 436M/436M [00:08<00:00, 51.7MB/s] 
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassificatio

In [28]:
model.config.num_labels

7

In [29]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/MariaFogh/bert-finetuned-ner into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7500


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.285000,0.283911,0.800523,0.822423,0.811325,0.920102
2,0.205200,0.259886,0.802844,0.838258,0.820169,0.925432
3,0.141900,0.314129,0.816875,0.843843,0.830140,0.927475


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to bert-finetuned-ner/checkpoint-2500
Configuration saved in bert-finetuned-ner/checkpoint-2500/config.json
Model weights saved in bert-finetuned-ner/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-2500/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-2500/special_tokens_map.json
tokenizer config file saved in bert-finetuned-ner/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to bert-finetuned-ner/checkpoint-5000
Configuration saved in bert-finetuned-ner/checkpoint-5000/config.json
Model weights saved in bert-finetuned-ner/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-5000/tokenizer_config.json
Special tokens file saved in bert-

TrainOutput(global_step=7500, training_loss=0.23427290547688803, metrics={'train_runtime': 803.8838, 'train_samples_per_second': 74.638, 'train_steps_per_second': 9.33, 'total_flos': 792573386169312.0, 'train_loss': 0.23427290547688803, 'epoch': 3.0})

In [37]:
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to bert-finetuned-ner
Configuration saved in bert-finetuned-ner/config.json
Model weights saved in bert-finetuned-ner/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/411M [00:00<?, ?B/s]

Upload file runs/Oct11_11-44-27_473846f0cc56/events.out.tfevents.1665488678.473846f0cc56.68.0:  42%|####2     …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/MariaFogh/bert-finetuned-ner
   c637c59..c4a2fcc  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/MariaFogh/bert-finetuned-ner
   c637c59..c4a2fcc  main -> main

To https://huggingface.co/MariaFogh/bert-finetuned-ner
   c4a2fcc..06ce4a8  main -> main

   c4a2fcc..06ce4a8  main -> main



'https://huggingface.co/MariaFogh/bert-finetuned-ner/commit/c4a2fcccf9962152191a51f5d130d49dea071aba'

In [38]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [39]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absol

In [40]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [41]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [42]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [43]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'MariaFogh/bert-finetuned-ner-accelerate'

In [44]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/MariaFogh/bert-finetuned-ner-accelerate into local empty directory.


In [45]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [46]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/7500 [00:00<?, ?it/s]

Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 0: {'precision': 0.8187473490739432, 'recall': 0.7617732175743226, 'f1': 0.7892333901192504, 'accuracy': 0.9157279238369086}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 1: {'precision': 0.8326735472925209, 'recall': 0.7964703495841504, 'f1': 0.8141696906860205, 'accuracy': 0.9238193756520062}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Several commits (2) will be pushed upstream.
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 2: {'precision': 0.8396719920825675, 'recall': 0.8016467571033272, 'f1': 0.8202188999758312, 'accuracy': 0.9262652207835471}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Several commits (3) will be pushed upstream.


In [47]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

Configuration saved in bert-finetuned-ner-accelerate/config.json
Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin


In [49]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "MariaFogh/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--MariaFogh--bert-finetuned-ner/snapshots/06ce4a854b66747c7093392c633d9cf32c4c6e7c/config.json
Model config BertConfig {
  "_name_or_path": "MariaFogh/bert-finetuned-ner",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--MariaFogh--bert-finetuned-ner/snapshots/06ce4a854b66747c7093392c633d9cf32c4c6e7c/pytorch_model.bin
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint at MariaFogh/bert-finetuned-ner.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.


Downloading:   0%|          | 0.00/347 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--MariaFogh--bert-finetuned-ner/snapshots/06ce4a854b66747c7093392c633d9cf32c4c6e7c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--MariaFogh--bert-finetuned-ner/snapshots/06ce4a854b66747c7093392c633d9cf32c4c6e7c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--MariaFogh--bert-finetuned-ner/snapshots/06ce4a854b66747c7093392c633d9cf32c4c6e7c/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--MariaFogh--bert-finetuned-ner/snapshots/06ce4a854b66747c7093392c633d9cf32c4c6e7c/tokenizer_config.json


[{'entity_group': 'PER',
  'score': 0.98081625,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9869027,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.99057686,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]